### Implementación Greedy-Prim

Autor: Diego Osorio

In [1]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
from random import sample, randint
import math
import numpy as np
sys.path.insert(1, 'util/')
plt.rcParams['figure.figsize'] = (10,10)

In [2]:
# Warehouse points
wh = pd.read_csv('../DataSet/Almacenes.csv').values.tolist()
# Delivery points
dp = pd.read_csv('../DataSet/PuntosEntrega.csv').values.tolist()

distance = lambda p1, p2: abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

#### Descomposition
Recibe `total` y `parts` y retorna una lista (`stocks`) con `parts` valores aleatorios que sumados dan `total` \
$\sum_{i=0}^{len(parts)}  $ `stocks[i]` $ = $ `total`

In [3]:
def decomposition(total, parts):
    stocks = []
    consumed = 0
    for _ in range(parts):
        curr = randint((total//parts)-randint(0, 15), (total//parts)+randint(0,15))
        curr = abs(curr)
        if curr + consumed < total:
            consumed += curr
            stocks.append(curr)
        else:
            stocks.append(total-consumed)
            consumed += (total - consumed)
    st = sum(stocks)
    print(stocks)
    if st < total:
        stocks[-1] = stocks[-1] + (total - st)
    return stocks

#### Heuristica
Se tomara como heuristica para cada punto de delivery su almacen mas cercano. Con la intencion de que cada vez que el vehiculo se quede sin paquetes en la ruta, se desvie al almacen mas cercano. \
`heuristica[almacen] = indice_almacen` 

In [4]:
hr = [] 
for d in dp:
    cw = min(wh, key=lambda w: distance(w, d))
    hr += [wh.index(cw)]

#### Información del almacen
Se generará una cantidad aleatoria de paquetes que será mayor a la cantidad de puntos de entrega.
Se distribuirán los paquetes en cada almacen en partes aleatorias con la funcion `descomposition`

In [5]:
total_packages = randint(len(dp), len(dp) + randint(5, 100))
print('Total packages:', total_packages)
wh_stock = decomposition(total_packages, len(wh))

Total packages: 5023
[52, 47, 50, 53, 52, 49, 48, 48, 45, 40, 47, 63, 53, 59, 49, 49, 43, 51, 52, 52, 57, 54, 47, 38, 56, 47, 53, 55, 50, 57, 56, 41, 50, 44, 50, 46, 61, 40, 52, 44, 54, 50, 49, 46, 57, 49, 42, 63, 49, 43, 52, 51, 41, 57, 47, 41, 51, 52, 42, 58, 42, 37, 53, 50, 59, 45, 41, 48, 54, 57, 55, 44, 46, 43, 54, 48, 57, 50, 43, 55, 51, 50, 46, 60, 48, 50, 51, 46, 59, 44, 55, 60, 45, 53, 48, 43, 55, 53, 58, 42]


Se genrará un dataframe del cual sacaremos la información en la ejecución del algoritmo.
Guardaremos la posición de todos los `almacenes`, los `paquetes` que deben entregar y la cantidad de puntos de entrega que lo consideran el `almacén` mas cercano según la `heuristica`

In [6]:
data = {'Position': [], 'Close DP\'s': [], 'Packages to give': []}
for i in range(len(wh)):
    data['Position'].append(f'({wh[i][0]}, {wh[i][1]})')
    data['Close DP\'s'].append(len([h for h in hr if h == i]))
    data['Packages to give'].append(wh_stock[i])
df = pd.DataFrame(data=data)
print('Total packages:', total_packages)
ignore = []
start = np.argmax(df['Packages to give'])
print('Starting point:', start)
df

Total packages: 5023
Starting point: 99


,Position,Close DP's,Packages to give
0,"(672, 301)",53,52
1,"(811, 500)",93,47
2,"(959, 717)",37,50
3,"(140, 368)",62,53
4,"(609, 743)",22,52
...,...,...,...
95,"(31, 417)",78,43
96,"(615, 785)",39,55
97,"(555, 236)",23,53
98,"(265, 437)",44,58


## Algorithm 
El algoritmo tomará como punto de inicio el almacen con más puntos cercano.
Se realizará una implementación naive del algoritmo de Prim.

Para ahorrar memoria y no usar un arreglo de visitados, se borrarán los puntos ya visitados del arreglo existente en la funcion `get_next_point` que recibe un arreglo de puntos y retorna el más cercano al actual, eliminando este del arreglo. 

In [7]:
def get_next_point(current, posibilities):
    n = min(posibilities, key=lambda p: distance(current, p))
    ind = posibilities.index(n)
    del posibilities[ind]
    return n, ind

El algoritmo $O(n^2)$, consiste en recorrer todos los puntos de entrega y cada vez dirigirse al siguiente nodo cuya arista tenga el menor peso. Parecido al algoritmo de Prim. 
Cuando el vehiculo entregue todos los paquetes que tiene, se dirigirá al warehouse más cercano y seguirá con la ruta.

In [10]:
dpcopy = dp.copy()
whcopy = wh.copy()
c_w = whcopy[start]
route = [c_w]
c_l = df['Packages to give'][start]
print(f'Warehouse #{start}: Recogio {c_l} paquetes',end='\n\t')
for i in range(len(dp)):
    if c_l == 0:
        c_w, w_i = get_next_point(route[-1], whcopy)
        route += [c_w]
        c_l = df['Packages to give'][w_i]
        print(f'\nWarehouse #{w_i}: Recogio {c_l} paquetes',end='\n\t')
        
    route += [get_next_point(route[-1], dpcopy)[0]]
    print('. ', end='')
    c_l -= 1
print()
print('Nodos recorridos:', len(route))

Warehouse #99: Recogio 73 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #3: Recogio 53 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #42: Recogio 49 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #6: Recogio 48 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #74: Recogio 54 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #46: Recogio 42 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #25: Recogio 47 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #

	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #6: Recogio 48 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #33: Recogio 44 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #19: Recogio 52 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #20: Recogio 57 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #11: Recogio 63 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #2: Recogio 50 paquetes
	. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Warehouse #5: Recogio 49 paquetes
	. . . . . .

### Mejora
Más vehículos para minimizar las rutas largas a la hora de volver a un almacén.